In [17]:
# USER OPTIONS
# define tokenizer name - should be one among the keys in the cell below
tokenizer_name = 'ChordSymbolTokenizer' # or any other name from the keys in tokenizers dictionary
# tokenizer_name = 'RootTypeTokenizer'
# tokenizer_name = 'PitchClassTokenizer'
# folder to xmls
val_dir = '/media/maindisk/maximos/data/hooktheory_test'
# val_dir = '/media/maindisk/maximos/data/gjt_melodies/Library_melodies'
# val_dir = '/media/datadisk/datasets/gjt_melodies/Library_melodies'
# val_dir = '/media/maximos/9C33-6BBD/data/gjt_melodies/Library_melodies'

# define batch size depending on GPU availability / status
batchsize = 16
# select device name - could be 'cpu', 'cuda', 'coda:0', 'cuda:1'...
device_name = 'cpu'

In [18]:
from data_utils import SeparatedMelHarmDataset
import os
import numpy as np
from harmony_tokenizers_m21 import ChordSymbolTokenizer, RootTypeTokenizer, \
    PitchClassTokenizer, RootPCTokenizer, GCTRootPCTokenizer, \
    GCTSymbolTokenizer, GCTRootTypeTokenizer, MelodyPitchTokenizer, \
    MergedMelHarmTokenizer
from torch.utils.data import DataLoader
from transformers import BartForConditionalGeneration, BartConfig, DataCollatorForSeq2Seq
import torch
from tqdm import tqdm

In [19]:
tokenizers = {
    'ChordSymbolTokenizer': ChordSymbolTokenizer,
    'RootTypeTokenizer': RootTypeTokenizer,
    'PitchClassTokenizer': PitchClassTokenizer,
    'RootPCTokenizer': RootPCTokenizer,
    'GCTRootPCTokenizer': GCTRootPCTokenizer,
    'GCTSymbolTokenizer': GCTSymbolTokenizer,
    'GCTRootTypeTokenizer': GCTRootTypeTokenizer
}

In [20]:
melody_tokenizer = MelodyPitchTokenizer.from_pretrained('saved_tokenizers/MelodyPitchTokenizer')
harmony_tokenizer = tokenizers[tokenizer_name].from_pretrained('saved_tokenizers/' + tokenizer_name)

tokenizer = MergedMelHarmTokenizer(melody_tokenizer, harmony_tokenizer)

model_path = 'saved_models/bart/' + tokenizer_name + '/' + tokenizer_name + '.pt'

In [21]:
bart_config = BartConfig(
    vocab_size=len(tokenizer.vocab),
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    decoder_start_token_id=tokenizer.bos_token_id,
    forced_eos_token_id=tokenizer.eos_token_id,
    max_position_embeddings=512,
    encoder_layers=8,
    encoder_attention_heads=8,
    encoder_ffn_dim=512,
    decoder_layers=8,
    decoder_attention_heads=8,
    decoder_ffn_dim=512,
    d_model=512,
    encoder_layerdrop=0.3,
    decoder_layerdrop=0.3,
    dropout=0.3
)

model = BartForConditionalGeneration(bart_config)

checkpoint = torch.load(model_path, map_location="cpu", weights_only=True)
model.load_state_dict(checkpoint)

model.eval()

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(545, 512, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(545, 512, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(514, 512)
      (layers): ModuleList(
        (0-7): 8 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=512, bias=True)
          (fc2): Linear(in_features=512, out_features=512, bias=True)
          (final_layer_norm

In [22]:
val_dataset = SeparatedMelHarmDataset(val_dir, tokenizer, max_length=512, num_bars=64)

def create_data_collator(tokenizer, model):
    return DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

collator = create_data_collator(tokenizer, model=model)

valloader = DataLoader(val_dataset, batch_size=batchsize, shuffle=True, collate_fn=collator)

In [23]:
if device_name == 'cpu':
    device = torch.device('cpu')
else:
    if torch.cuda.is_available():
        device = torch.device(device_name)
    else:
        print('Selected device not available: ' + device_name)
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(545, 512, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(545, 512, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(514, 512)
      (layers): ModuleList(
        (0-7): 8 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=512, out_features=512, bias=True)
          (fc2): Linear(in_features=512, out_features=512, bias=True)
          (final_layer_norm

In [24]:
with torch.no_grad():
    with tqdm(valloader, unit='batch') as tepoch:
        tepoch.set_description(f'Running')
        print(tepoch)
        for batch in tepoch:
            input_tokens = []
            real_tokens = []
            output_tokens = []
            for bi in range( len(batch['input_ids']) ):
                input_ids = batch['input_ids'][bi].to(device)
                real_ids = batch['labels'][bi]
                for i in input_ids:
                    input_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
                print('input_tokens: ', input_tokens)

                for i in real_ids:
                    if i > 0:
                        real_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
                print('real_tokens: ', real_tokens)
                
                outputs = model.generate(
                    input_ids=input_ids.reshape(1, input_ids.shape[0]),
                    # attention_mask=batch['attention_mask'][bi],
                    eos_token_id=tokenizer.eos_token_id,
                    max_new_tokens=500,
                    do_sample=False,
                    temperature=1.0
                )
                for i in outputs[0]:
                    output_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
                print('output_tokens: ', output_tokens)
                break
            break

Running:   0%|          | 0/95 [00:00<?, ?batch/s]

Running:   0%|          | 0/95 [00:00<?, ?batch/s]
input_tokens:  ['<s>', '<bar>', 'ts_4x4', 'position_0x00', '<rest>', 'position_1x00', 'P:60', 'position_1x25', 'P:62', 'position_1x50', 'P:64', 'position_1x75', 'P:60', 'position_2x00', 'P:62', 'position_2x25', 'P:67', 'position_3x00', '<rest>', '<bar>', 'position_0x00', '<rest>', 'position_1x00', 'P:65', 'position_1x25', 'P:65', 'position_1x50', 'P:65', 'position_1x75', 'P:65', 'position_2x00', 'P:64', 'position_2x75', '<rest>', 'position_3x75', '<rest>', '<bar>', 'position_0x00', '<rest>', 'position_1x00', 'P:60', 'position_1x25', 'P:62', 'position_1x50', 'P:64', 'position_1x75', 'P:60', 'position_2x00', 'P:62', 'position_2x25', 'P:62', 'position_2x50', 'P:67', 'position_3x25', '<rest>', '<bar>', 'position_0x00', '<rest>', 'position_1x00', 'P:65', 'position_1x25', 'P:65', 'position_1x50', 'P:65', 'position_1x75', 'P:65', 'position_2x00', 'P:64', 'position_2x75', '<rest>', 'position_3x75', '<rest>', '<bar>', 'position_0x00', 'P:64', '

Running:   0%|          | 0/95 [00:00<?, ?batch/s]

output_tokens:  ['<s>', '<h>', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'F:maj', '<bar>', 'position_0x00', 'C:maj', '<bar>', 'position_0x00', 'F:maj', '</s>']
